<a href="https://colab.research.google.com/github/Hisernberg/Ai-agentic-projects/blob/main/Copy_of_Image_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh
!nohup ollama serve > output.log 2>&1 &
!ollama pull phi4-mini

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.



In [ ]:
!pip install opencv-python pillow requests langchain langchain_community ollama langchain_ollama requests pillow torchvision torch transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.9/437.9 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4

In [ ]:
# ✅ Download Sample Images
import requests
from pathlib import Path
from PIL import Image
from torchvision import models, transforms
import torch

Path("sample_images").mkdir(exist_ok=True)
images = {
    "dog.jpg": "https://images.unsplash.com/photo-1518717758536-85ae29035b6d",
    "cat.jpg": "https://images.unsplash.com/photo-1592194996308-7b43878e84a6",
    "car.jpg": "https://images.unsplash.com/photo-1503376780353-7e6692767b70"
}
for filename, url in images.items():
    response = requests.get(url)
    with open(f"sample_images/{filename}", "wb") as f:
        f.write(response.content)
print("✅ Sample images downloaded.")

# ✅ Load Pre-trained Model
model = models.resnet50(pretrained=True)
model.eval()

LABELS_URL = "https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt"
labels = requests.get(LABELS_URL).text.strip().split("\n")
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
def recognize_image(image_path):
    img = Image.open(image_path).convert("RGB")
    input_tensor = transform(img).unsqueeze(0)
    with torch.no_grad():
        outputs = model(input_tensor)
    _, predicted_idx = torch.max(outputs, 1)
    return labels[predicted_idx.item()]

# ✅ Setup LangChain with Ollama (Phi-4 Mini)
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = Ollama(model="phi4-mini")  # Make sure Ollama is running locally with `phi4-mini` pulled
template = """
You are an AI assistant helping with image recognition.
The image content was identified as: {description}

Please describe this object and suggest possible one short use cases.

Answer:
"""
prompt = PromptTemplate.from_template(template)
chain = LLMChain(llm=llm, prompt=prompt)

# ✅ Process Images
for image_name in images:
    image_path = f"sample_images/{image_name}"
    label = recognize_image(image_path)
    print(f"\n🖼️ Image: {image_name}")
    print(f"📷 Recognized Label: {label}")
    response = chain.run({"description": label})
    print(f"🤖 Phi-4 Mini Response:\n{response}")


✅ Sample images downloaded.


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



🖼️ Image: dog.jpg
📷 Recognized Label: German short-haired pointer
🤖 Phi-4 Mini Response:
A German Short-Haired Pointer is a medium-sized dog known for its wiry, dense coat that ranges from light to dark red-brown. This breed has strong hindquarters with well-muscled legs giving it an athletic build suitable mainly as hunting dogs due to their high energy levels and excellent tracking abilities.

Possible use case: A German Short-Haired Pointer can be used in controlled environments for pest control, targeting rodents or raccoons that might invade agricultural areas.

🖼️ Image: cat.jpg
📷 Recognized Label: Persian cat
🤖 Phi-4 Mini Response:
A Persian Cat is a breed of domestic cats known for its long, luxurious fur that reaches down to the paws. It has distinct physical characteristics such as large eyes set far apart with almond-shaped pupils, an elongated body shape often described in terms like 'bat-like', and muscular build which contributes to their expressive facial features.

Pos

In [ ]:
!pip install transformers -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 102.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.3/489.3 kB 35.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yolov5 7.0.14 requires huggingface-hub<0.25.0,>=0.12.0, but you have huggingface-hub 0.31.4 which is incompatible.


In [ ]:
# Download sample images
import requests
from pathlib import Path
Path("sample_images").mkdir(exist_ok=True)

images = {
    "dog.jpg": "https://images.unsplash.com/photo-1518717758536-85ae29035b6d",
    "cat.jpg": "https://images.unsplash.com/photo-1592194996308-7b43878e84a6",
    "car.jpg": "https://images.unsplash.com/photo-1503376780353-7e6692767b70"
}

for filename, url in images.items():
    response = requests.get(url)
    with open(f"sample_images/{filename}", "wb") as f:
        f.write(response.content)
print("✅ Sample images downloaded.")

# Load CLIP model
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Define candidate labels
labels = ["dog", "cat", "car", "tree", "person", "computer", "building", "bottle", "phone", "book"]

def recognize_clip(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(text=labels, images=image, return_tensors="pt", padding=True).to(device)
    outputs = model(**inputs)
    logits_per_image = outputs.logits_per_image
    probs = logits_per_image.softmax(dim=1)
    predicted_index = probs.argmax().item()
    return labels[predicted_index]

# LangChain + Ollama (Requires local Ollama server with `phi4-mini`)
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = Ollama(model="phi4-mini")  # Requires Ollama running locally
prompt = PromptTemplate.from_template("""
You are an AI assistant helping with image recognition.
The image content was identified as: {description}

Please describe this object and suggest possible use cases.

Answer:
""")
chain = LLMChain(llm=llm, prompt=prompt)

# Run recognition and generate response
for image_name in images:
    image_path = f"sample_images/{image_name}"
    label = recognize_clip(image_path)
    print(f"\n🖼️ Image: {image_name}")
    print(f"📷 Recognized Label (CLIP): {label}")
    response = chain.run({"description": label})
    print(f"🤖 Phi-4 Mini Response:\n{response}")


✅ Sample images downloaded.

🖼️ Image: dog.jpg
📷 Recognized Label (CLIP): dog
🤖 Phi-4 Mini Response:
A "dog" is a domesticated carnivorous mammal belonging to the Canidae family. Dogs typically have four legs, fur covering their bodies which can vary in color from white through shades of grey or black, with some having markings such as spots; they possess ears that are usually pointed and eyes positioned on either side at different heights depending upon breed (e.g., brachycephalic breeds like bulldogs will naturally tilt the head forward). Their noses have a wet nose area called the muzzle. Dogs also exhibit a wide range of behaviors due to their intelligence, such as barking or vocalizing for communication.

Possible use cases/suggestions:

1. **Companionship:** One primary role that dogs play is serving as pets and companions; they offer emotional support through affection which can reduce stress levels in humans.
   
2. **Security/Guardianship:** Dogs have been trained to work with

In [ ]:
!pip install -U huggingface_hub

In [ ]:
import requests
from PIL import Image
from io import BytesIO
import torch
from transformers import CLIPProcessor, CLIPModel
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
import os
import numpy as np

# Download sample images
def download_sample_images():
    image_urls = [
        "https://images.unsplash.com/photo-1600585154340-be6161a56a0c",  # House
        "https://images.unsplash.com/photo-1518791841217-8f162f1e1131",  # Cat
        "https://images.unsplash.com/photo-1507525428034-b723cf961d3e"   # Beach
    ]
    image_paths = []
    captions = [
        "A modern house with large windows",
        "A fluffy cat sitting on a couch",
        "A tropical beach with palm trees"
    ]
    os.makedirs("sample_images", exist_ok=True)

    for i, url in enumerate(image_urls):
        try:
            response = requests.get(url)
            if response.status_code == 200:
                img = Image.open(BytesIO(response.content))
                img_path = f"sample_images/image_{i+1}.jpg"
                img.save(img_path)
                image_paths.append(img_path)
                print(f"Downloaded image {i+1} to {img_path}")
            else:
                print(f"Failed to download image {i+1}")
        except Exception as e:
            print(f"Error downloading image {i+1}: {e}")

    return image_paths, captions

# Initialize CLIP and Ollama
def initialize_models():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
    clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    llm = Ollama(model="phi4-mini")
    return clip_model, clip_processor, llm, device

# Use Case 1: Image Recognition (classify image based on text labels)
def image_recognition(image_path, labels, clip_model, clip_processor, device):
    image = Image.open(image_path).convert("RGB")
    inputs = clip_processor(text=labels, images=image, return_tensors="pt", padding=True).to(device)

    with torch.no_grad():
        outputs = clip_model(**inputs)
        logits_per_image = outputs.logits_per_image
        probs = logits_per_image.softmax(dim=1).cpu().numpy()[0]

    result = {label: prob for label, prob in zip(labels, probs)}
    predicted_label = max(result, key=result.get)
    return predicted_label, result

# Use Case 2: Image Description (generate detailed description using Phi-4 Mini)
def image_description(image_path, clip_model, clip_processor, llm, device):
    image = Image.open(image_path).convert("RGB")
    # Use CLIP to get a basic classification to guide the description
    labels = ["house", "cat", "beach", "car", "tree"]
    inputs = clip_processor(text=labels, images=image, return_tensors="pt", padding=True).to(device)

    with torch.no_grad():
        outputs = clip_model(**inputs)
        logits_per_image = outputs.logits_per_image
        probs = logits_per_image.softmax(dim=1).cpu().numpy()[0]
        top_label = labels[np.argmax(probs)]

    # Create a prompt for Phi-4 Mini
    prompt_template = PromptTemplate(
        input_variables=["label"],
        template="Generate a detailed description of an image that primarily features a {label}. Include details about possible colors, setting, and context."
    )
    prompt = prompt_template.format(label=top_label)

    try:
        description = llm.invoke(prompt)
        return description
    except Exception as e:
        return f"Error generating description: {e}"

# Use Case 3: Image Search by Text (find image matching a text query)
def image_search_by_text(text_query, image_paths, clip_model, clip_processor, device):
    images = [Image.open(path).convert("RGB") for path in image_paths]
    inputs = clip_processor(text=[text_query], images=images, return_tensors="pt", padding=True).to(device)

    with torch.no_grad():
        outputs = clip_model(**inputs)
        logits_per_image = outputs.logits_per_image
        probs = logits_per_image.softmax(dim=1).cpu().numpy()[0]

    best_image_idx = np.argmax(probs)
    return image_paths[best_image_idx], probs[best_image_idx]

# Use Case 4: Image Search by Image (find most similar image)
def image_search_by_image(query_image_path, image_paths, clip_model, clip_processor, device):
    query_image = Image.open(query_image_path).convert("RGB")
    images = [Image.open(path).convert("RGB") for path in image_paths]

    # Encode query image
    query_inputs = clip_processor(images=query_image, return_tensors="pt").to(device)
    with torch.no_grad():
        query_features = clip_model.get_image_features(**query_inputs).cpu().numpy()

    # Encode all images
    image_features = []
    for image in images:
        inputs = clip_processor(images=image, return_tensors="pt").to(device)
        with torch.no_grad():
            features = clip_model.get_image_features(**inputs).cpu().numpy()
        image_features.append(features)

    # Compute cosine similarities
    similarities = [
        np.dot(query_features, feat.T) / (np.linalg.norm(query_features) * np.linalg.norm(feat))
        for feat in image_features
    ]

    best_image_idx = np.argmax(similarities)
    return image_paths[best_image_idx], similarities[best_image_idx]

def main():
    # Download sample images
    image_paths, captions = download_sample_images()

    # Initialize models
    clip_model, clip_processor, llm, device = initialize_models()

    # Use Case 1: Image Recognition
    print("\n=== Image Recognition ===")
    labels = ["a house", "a cat", "a beach", "a car", "a tree"]
    for i, image_path in enumerate(image_paths):
        predicted_label, probs = image_recognition(image_path, labels, clip_model, clip_processor, device)
        print(f"Image {i+1} ({image_path}):")
        print(f"Predicted: {predicted_label}")
        print(f"Probabilities: {probs}")

    # Use Case 2: Image Description
    print("\n=== Image Description ===")
    for i, image_path in enumerate(image_paths):
        description = image_description(image_path, clip_model, clip_processor, llm, device)
        print(f"Image {i+1} ({image_path}):")
        print(f"Description: {description}")

    # Use Case 3: Image Search by Text
    print("\n=== Image Search by Text ===")
    text_query = "A tropical beach with palm trees"
    best_image, score = image_search_by_text(text_query, image_paths, clip_model, clip_processor, device)
    print(f"Text Query: '{text_query}'")
    print(f"Best Match: {best_image} (Score: {score:.4f})")

    # Use Case 4: Image Search by Image
    print("\n=== Image Search by Image ===")
    query_image_path = image_paths[0]  # Use first image as query
    best_match, similarity = image_search_by_image(query_image_path, image_paths, clip_model, clip_processor, device)
    print(f"Query Image: {query_image_path}")
    print(f"Best Match: {best_match} (Similarity: {float(similarity):.4f})")

if __name__ == "__main__":
    main()

Downloaded image 1 to sample_images/image_1.jpg
Downloaded image 2 to sample_images/image_2.jpg
Downloaded image 3 to sample_images/image_3.jpg

=== Image Recognition ===
Image 1 (sample_images/image_1.jpg):
Predicted: a house
Probabilities: {'a house': np.float32(0.9935417), 'a cat': np.float32(0.000102968894), 'a beach': np.float32(7.310011e-05), 'a car': np.float32(0.00032168103), 'a tree': np.float32(0.0059605595)}
Image 2 (sample_images/image_2.jpg):
Predicted: a cat
Probabilities: {'a house': np.float32(0.00063693087), 'a cat': np.float32(0.9987239), 'a beach': np.float32(4.9661146e-05), 'a car': np.float32(0.00033760496), 'a tree': np.float32(0.00025181915)}
Image 3 (sample_images/image_3.jpg):
Predicted: a beach
Probabilities: {'a house': np.float32(0.0013106146), 'a cat': np.float32(0.00016481701), 'a beach': np.float32(0.9975725), 'a car': np.float32(0.0004518284), 'a tree': np.float32(0.000500347)}

=== Image Description ===
Image 1 (sample_images/image_1.jpg):
Description: 

<ipython-input-11-23a04966a4e6>:165: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(f"Best Match: {best_match} (Similarity: {float(similarity):.4f})")
